[![Open In Colab](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/badge/open-in-colab.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/structural-break/quickstarters/baseline/baseline.ipynb)
[![Open In Kaggle](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/badge/open-in-kaggle.svg)](https://www.kaggle.com/code/crunchdao/structural-break-baseline)

In [1]:
%pip install crunch-cli --upgrade --quiet --progress-bar off
!crunch setup-notebook structural-break MOc241YzAb1h8V4n463c9UXp

crunch-cli, version 7.4.0
main.py: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/submissions/23863/main.py (6729 bytes)
requirements.txt: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/submissions/23863/requirements.original.txt (194 bytes)
resources/cnnlstm_model.py: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/models/25094/cnnlstm_model.py (19374 bytes)
resources/lgbm_model.py: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/models/25094/lgbm_model.py (17887 bytes)
resources/lightgbm_features_cache.pkl: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/models/25094/lightgbm_features_cache.pkl (5784251 bytes)
resources/tft_model.py: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/models/25094/tft_model.py (20804 bytes)
data/X_train.parquet: download from https:crunchdao--com

In [2]:
import os
import typing

# Import your dependencies
import joblib
import pandas as pd
import scipy
import sklearn.metrics

In [3]:
import crunch

# Load the Crunch Toolings
crunch = crunch.load_notebook()

loaded inline runner with module: <module '__main__'>

cli version: 7.4.0
available ram: 12.67 gb
available cpu: 2 core
----


In [4]:
pip install autocpd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 120.8 MB/s eta 0:00:00


In [6]:
pip install tensorflow_docs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.7/182.7 kB 4.8 MB/s eta 0:00:00


In [12]:
import numpy as np
import tensorflow as tf
import tensorflow_docs.modeling as tfdoc_model
import tensorflow_docs.plots as tfdoc_plot
from sklearn.model_selection import train_test_split
from autocpd.neuralnetwork import compile_and_fit, general_simple_nn,general_deep_nn  # *** Changed to simple_nn ***
from autocpd.utils import Transform2D2TR
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
import numpy as np


In [13]:
# %%
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import tensorflow_docs.modeling as tfdoc_model
import tensorflow_docs.plots as tfdoc_plot
from sklearn.model_selection import train_test_split

from autocpd.neuralnetwork import compile_and_fit, general_deep_nn
from autocpd.utils import DataSetGen, Transform2D2TR

In [10]:
"""
Time Series Data Loading and Iteration Utilities

This module provides utilities for loading and iterating through time series data
with boundary point detection and cropping functionality.
"""

import pandas as pd
import numpy as np
from typing import Iterator, Tuple, Optional, Dict, Any
import os
import warnings
warnings.filterwarnings('ignore')


class TimeSeriesDataLoader:
    """
    A utility class for loading and iterating through time series data.

    This class handles:
    - Loading multiindex time series data from parquet files
    - Iterating through individual time series
    - Boundary point detection (where 'period' changes from 0 to 1)
    - Cropping time series around boundary points
    """

    def __init__(self,
                 x_path: str = "data/X_train.parquet",
                 y_path: str = "data/y_train.parquet",
                 window_size: int = 249,
                 keep_period: bool = True):
        """
        Initialize the TimeSeriesDataLoader.

        Args:
            x_path: Path to the X data parquet file
            y_path: Path to the y data parquet file
            window_size: Number of timesteps before and after boundary point to include
        """
        self.x_path = x_path
        self.y_path = y_path
        self.window_size = window_size
        self.keep_period = keep_period
        # Load data
        self._load_data()

    def _load_data(self):
        """Load the data from parquet files."""
        print(f"Loading X data from {self.x_path}...")
        self.X = pd.read_parquet(self.x_path)

        print(f"Loading y data from {self.y_path}...")
        self.y = pd.read_parquet(self.y_path)

        # Ensure X has proper MultiIndex
        if not isinstance(self.X.index, pd.MultiIndex):
            if 'id' in self.X.columns and 'time' in self.X.columns:
                self.X.set_index(['id', 'time'], inplace=True)
            else:
                # Try to infer from index structure
                if hasattr(self.X.index, 'names') and len(self.X.index.names) == 2:
                    self.X.index.names = ['id', 'time']
                else:
                    raise ValueError("X data must have MultiIndex with 'id' and 'time' levels")

        # Process y data
        if "structural_breakpoint" in self.y.columns:
            self.y = self.y["structural_breakpoint"]
        else:
            self.y = self.y.squeeze()

        # Get unique series IDs
        self.series_ids = self.X.index.get_level_values('id').unique()
        print(f"Loaded {len(self.series_ids)} time series")

    def get_series_iterator(self,
                          crop_around_boundary: bool = False,
                          include_y: bool = True) -> Iterator[Tuple[str, pd.DataFrame, Optional[float]]]:
        """
        Get an iterator that yields individual time series.

        Args:
            crop_around_boundary: If True, crop each series around its boundary point
            include_y: If True, include the corresponding y value

        Yields:
            Tuple of (series_id, series_data, y_value)
        """
        for series_id in self.series_ids:
            series_data = self.X.loc[series_id].copy()

            if crop_around_boundary:
                series_data = self._crop_around_boundary(series_id, series_data)
                if series_data is None:
                    continue  # Skip series that couldn't be cropped

            y_value = self.y.loc[series_id] if include_y else None
            if self.keep_period == False:
                series_data = series_data.drop("period",axis=1)
            yield series_id, series_data, y_value

    def _find_boundary_point(self, series_data: pd.DataFrame) -> Optional[int]:
        """
        Find the boundary point in a time series.

        Args:
            series_data: DataFrame containing the time series data

        Returns:
            Index of the boundary point, or None if not found
        """
        if 'period' not in series_data.columns:
            return None

        period = series_data['period'].values

        # Find the first index where period goes from 0 to 1
        bp_candidates = np.where((period[:-1] == 0) & (period[1:] == 1))[0]

        if len(bp_candidates) == 0:
            # Fallback: just where period == 1
            bp_candidates = np.where(period == 1)[0]

        if len(bp_candidates) == 0:
            return None

        # Return the first boundary point
        bp = bp_candidates[0] + 1 if (period[:-1] == 0).any() else bp_candidates[0]
        return bp

    def _crop_around_boundary(self, series_id: str, series_data: pd.DataFrame) -> Optional[pd.DataFrame]:
        """
        Crop a time series around its boundary point.

        Args:
            series_id: ID of the time series
            series_data: DataFrame containing the time series data

        Returns:
            Cropped DataFrame or None if cropping is not possible
        """
        bp = self._find_boundary_point(series_data)

        if bp is None:
            print(f"Warning: No boundary point found for series {series_id}")
            return None

        start = bp - self.window_size
        end = bp + self.window_size-1

        if start < 0 or end >= len(series_data):
            print(f"Warning: Series {series_id} too short for cropping (length: {len(series_data)}, required: {2*self.window_size + 1})")
            return None

        cropped_data = series_data.iloc[start:end+1].copy()

        # Reset the time index to start from 1
        cropped_data.index = pd.RangeIndex(1, len(cropped_data) + 1)

        return cropped_data

    def get_series_by_id(self, series_id: str, crop_around_boundary: bool = False) -> Tuple[pd.DataFrame, float]:
        """
        Get a specific time series by ID.

        Args:
            series_id: ID of the time series to retrieve
            crop_around_boundary: If True, crop around boundary point

        Returns:
            Tuple of (series_data, y_value)
        """
        if series_id not in self.series_ids:
            raise ValueError(f"Series ID {series_id} not found in dataset")

        series_data = self.X.loc[series_id].copy()
        if crop_around_boundary:
            boundary_point = self.window_size +1
        else:
            boundary_point = self._find_boundary_point(series_data)
        if crop_around_boundary:
            series_data = self._crop_around_boundary(series_id, series_data)
            if series_data is None:
                raise ValueError(f"Could not crop series {series_id} around boundary point")
        if self.keep_period == False:
                series_data = series_data.drop("period",axis=1)
        y_value = self.y.loc[series_id]
        return series_data, y_value, boundary_point

    def get_all_cropped_series(self) -> Tuple[pd.DataFrame, pd.Series]:
        """
        Get all time series cropped around their boundary points.

        Returns:
            Tuple of (X_cropped, y_cropped) where X_cropped is a MultiIndex DataFrame
        """
        cropped_series = []
        kept_ids = []

        for series_id, series_data, y_value in self.get_series_iterator(crop_around_boundary=True):
            cropped_data = self._crop_around_boundary(series_id, series_data)
            if cropped_data is not None:
                # Set proper MultiIndex for the cropped data
                cropped_data.index = pd.MultiIndex.from_product(
                    [[series_id], range(1, len(cropped_data) + 1)],
                    names=['id', 'time']
                )
                cropped_series.append(cropped_data)
                kept_ids.append(series_id)

        if not cropped_series:
            raise ValueError("No series could be cropped successfully")

        X_cropped = pd.concat(cropped_series)
        y_cropped = self.y.loc[kept_ids]
        if self.keep_period == False:
                X_cropped = X_cropped.drop("period",axis=1)
        print(f"Successfully cropped {len(kept_ids)} out of {len(self.series_ids)} series")

        return X_cropped, y_cropped

    def get_series_info(self) -> Dict[str, Any]:
        """
        Get information about the loaded dataset.

        Returns:
            Dictionary with dataset information
        """
        info = {
            'total_series': len(self.series_ids),
            'columns': list(self.X.columns),
            'total_timesteps': len(self.X),
            'avg_series_length': len(self.X) / len(self.series_ids),
            'window_size': self.window_size,
            'y_unique_values': self.y.unique().tolist(),
            'y_distribution': self.y.value_counts().to_dict()
        }
        return info

    def save_cropped_data(self,
                         x_output_path: str = "X_train_cropped.parquet",
                         y_output_path: str = "y_train_cropped.parquet"):
        """
        Save all cropped time series to parquet files.

        Args:
            x_output_path: Path to save cropped X data
            y_output_path: Path to save cropped y data
        """
        X_cropped, y_cropped = self.get_all_cropped_series()

        X_cropped.to_parquet(x_output_path)
        y_cropped.to_frame().to_parquet(y_output_path)

        print(f"Saved cropped data to {x_output_path} and {y_output_path}")

    def get_series_before_after_by_id(self, series_id: str, crop_around_boundary: bool = False) -> Tuple[pd.DataFrame, pd.DataFrame, float, int]:
        """
        Get a specific time series by ID, returning two series: before and after the boundary point.

        Args:
            series_id: ID of the time series to retrieve
            crop_around_boundary: If True, crop to window_size timesteps before and after boundary point

        Returns:
            Tuple of (series_before, series_after, y_value, boundary_point_index)
        """
        if series_id not in self.series_ids:
            raise ValueError(f"Series ID {series_id} not found in dataset")

        series_data = self.X.loc[series_id].copy()
        boundary_point = self._find_boundary_point(series_data)

        if boundary_point is None:
            raise ValueError(f"No boundary point found for series {series_id}")

        if crop_around_boundary:
            # Crop to window_size before and after boundary point
            start_before = max(0, boundary_point - self.window_size)
            end_before = boundary_point
            start_after = boundary_point
            end_after = min(len(series_data), boundary_point + self.window_size)
        else:
            # Use full series before and after boundary point
            start_before = 0
            end_before = boundary_point
            start_after = boundary_point
            end_after = len(series_data)

        # Extract before and after series
        series_before = series_data.iloc[start_before:end_before].copy()
        series_after = series_data.iloc[start_after:end_after].copy()

        # Reset indices to start from 1
        series_before.index = pd.RangeIndex(1, len(series_before) + 1)
        series_after.index = pd.RangeIndex(1, len(series_after) + 1)

        # Remove period column if not keeping it
        if self.keep_period == False:
            series_before = series_before.drop("period", axis=1)
            series_after = series_after.drop("period", axis=1)

        y_value = self.y.loc[series_id]
        return series_before, series_after, y_value, boundary_point


# Convenience functions for quick access
def load_timeseries_data(x_path: str = "data/X_train.parquet",
                        y_path: str = "data/y_train.parquet") -> TimeSeriesDataLoader:
    """
    Convenience function to create a TimeSeriesDataLoader instance.

    Args:
        x_path: Path to X data
        y_path: Path to y data

    Returns:
        TimeSeriesDataLoader instance
    """
    return TimeSeriesDataLoader(x_path, y_path)


def iterate_series(loader: TimeSeriesDataLoader,
                  crop_around_boundary: bool = False) -> Iterator[Tuple[str, pd.DataFrame, float]]:
    """
    Convenience function to iterate through all series.

    Args:
        loader: TimeSeriesDataLoader instance
        crop_around_boundary: Whether to crop around boundary points

    Yields:
        Tuple of (series_id, series_data, y_value)
    """
    for series_id, series_data, y_value in loader.get_series_iterator(crop_around_boundary=crop_around_boundary):
        yield series_id, series_data, y_value



In [14]:
all_series = []
all_labels = []
for i in range (0,10000,1):
  series_data,y_value,boundary = loader.get_series_by_id(i,True)
  series_data_clean = series_data["value"]
  all_series.append(series_data_clean)
  all_labels.append(y_value)
data_x = np.array(all_series)  # Shape: (num_ids, 498)
labels = np.array(all_labels)   # Shape: (num_ids,)

print(f"Data shape: {data_x.shape}")
print(f"Labels shape: {labels.shape}")

true_indices = np.where(labels == True)[0]
false_indices = np.where(labels == False)[0]

print(f"True samples: {len(true_indices)}")   # ~3333
print(f"False samples: {len(false_indices)}") # ~6666

from sklearn.utils import resample

false_balanced = resample(false_indices,
                         replace=False,
                         n_samples=len(true_indices),
                         random_state=42)

# Combine balanced indices
balanced_indices = np.concatenate([true_indices, false_balanced])
np.random.shuffle(balanced_indices)

# Create balanced dataset by indexing your existing arrays
data_x_balanced = data_x[balanced_indices]
labels_balanced = labels[balanced_indices].astype(int)  # Convert True/False to 1/0

print(f"Balanced data shape: {data_x_balanced.shape}")  # Should be (~6666, 498)
print(f"Balanced labels shape: {labels_balanced.shape}")
print(f"Balanced class distribution: {np.bincount(labels_balanced)}")

# Now use data_x_balanced and labels_balanced for your model
data_x = data_x_balanced
labels = labels_balanced

# Method 1: Per-sample min-max normalization (RECOMMENDED)
# This preserves the shape of each time series while normalizing to [0,1]
def normalize_per_sample(data):
    normalized = np.zeros_like(data)
    for i in range(data.shape[0]):
        sample = data[i]
        sample_min = sample.min()
        sample_max = sample.max()
        if sample_max > sample_min:  # Avoid division by zero
            normalized[i] = (sample - sample_min) / (sample_max - sample_min)
        else:
            normalized[i] = np.zeros_like(sample)  # Flat signal = all zeros
    return normalized

# Apply the better normalization
data_x_norm = normalize_per_sample(data_x_balanced)

# Check if this fixes the variation issue
print("After per-sample normalization:")
print(f"First sample std: {data_x_norm[0].std():.4f}")
print(f"Sample means: {data_x_norm.mean(axis=1)[:5]}")
print(f"Sample stds: {data_x_norm.std(axis=1)[:5]}")
print(f"Overall range: [{data_x_norm.min():.4f}, {data_x_norm.max():.4f}]")

# Split and train
x_train, x_test, y_train, y_test = train_test_split(
    data_x_norm, labels_balanced.reshape(-1, 1), train_size=0.8, random_state=42
)

x_train = Transform2D2TR(x_train, rescale=False, times=3)
x_test = Transform2D2TR(x_test, rescale=False, times=3)

Data shape: (10000, 498)
Labels shape: (10000,)
True samples: 2908
False samples: 7092
Balanced data shape: (5816, 498)
Balanced labels shape: (5816,)
Balanced class distribution: [2908 2908]
After per-sample normalization:
First sample std: 0.1193
Sample means: [0.64402393 0.44044606 0.42643469 0.44594874 0.39690033]
Sample stds: [0.11928432 0.18106374 0.11487443 0.06359554 0.08363304]
Overall range: [0.0000, 1.0000]


In [15]:
learning_rate = 1e-3
epochs = 100
batch_size = 64
dropout_rate = 0.3
n_filter = 16
n = x_train.shape[-1]
num_tran = x_train.shape[1]
kernel_size = (num_tran // 2, 10)
num_classes = 2
logdir = Path("tensorboard_logs", "Trial")
# %%
num_resblock = 3
model_name = "Toshko"
print(model_name)
# build the model
m = np.array([50, 40, 30, 20, 10])
model = general_deep_nn(
    n,
    num_tran,
    kernel_size,
    n_filter,
    dropout_rate,
    num_classes,
    num_resblock,
    m,
    5,
    model_name=model_name,
)
model.summary()

size_histories = {}
epochdots = tfdoc_model.EpochDots()
size_histories[model_name] = compile_and_fit(
    model,
    x_train,
    y_train,
    batch_size,
    learning_rate,
    model_name,
    logdir,
    epochdots,
    validation_split=0.25,
    max_epochs=epochs,
)
plotter = tfdoc_plot.HistoryPlotter(metric="accuracy", smoothing_std=10)
plt.figure(figsize=(10, 8))
plotter.plot(size_histories)
acc_name = model_name + "+acc.png"
acc_path = Path(logdir, model_name, acc_name)
plt.savefig(acc_path)


model_path = Path(logdir, model_name, "model")
model.save(model_path)

#  Confusion Matrix
model_pred = model.evaluate(x_test, y_test, verbose=2)
y_prob = np.max(model.predict(x_test), axis=1)
y_pred = np.argmax(model.predict(x_test), axis=1)
confusion_mtx = tf.math.confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
label_vec = np.arange(num_classes)
sns.heatmap(
    confusion_mtx,
    cmap="YlGnBu",
    xticklabels=label_vec,
    yticklabels=label_vec,
    annot=True,
    fmt="g",
)
plt.xlabel("Prediction")
plt.ylabel("Label")
cm_name = model_name + "Confusion_matrix.png"
cm_path = Path(logdir, model_name, cm_name)
plt.savefig(cm_path)

# save the confusion matrix
path_confusion_matrix = Path(logdir, model_name, "confusion_matrix")
np.save(path_confusion_matrix, confusion_mtx)

Toshko


Model: "Toshko"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input (InputLayer)  │ (None, 6, 498)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 6, 498, 1) │          0 │ Input[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 6, 498,    │         80 │ reshape[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 6, 498,    │         64 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 6, 498,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 3, 249,    │          0 │ re_lu[0][0]       │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 3, 249,    │      7,696 │ max_pooling2d[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 249,    │         64 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 3, 249,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 3, 249,    │      7,696 │ re_lu_1[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 249,    │         64 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 3, 249,    │          0 │ max_pooling2d[0]… │
│                     │ 16)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 3, 249,    │          0 │ add[0][0]         │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 3, 249,    │      7,696 │ re_lu_2[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 3, 249,    │         64 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 3, 249,    │          0 │ batch_normalizat… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 3, 249,    │      7,696 │ re_lu_3[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 51,676 (201.86 KB)

 Trainable params: 51,452 (200.98 KB)

 Non-trainable params: 224 (896.00 B)

Epoch 1/100

Epoch: 0, accuracy:0.4918,  loss:1.8557,  sparse_categorical_crossentropy:0.7044,  val_accuracy:0.4824,  val_loss:1.6157,  val_sparse_categorical_crossentropy:0.6933,  
.55/55 - 27s - 485ms/step - accuracy: 0.4918 - loss: 1.8557 - sparse_categorical_crossentropy: 0.7044 - val_accuracy: 0.4824 - val_loss: 1.6157 - val_sparse_categorical_crossentropy: 0.6933
Epoch 2/100
.55/55 - 22s - 396ms/step - accuracy: 0.5062 - loss: 1.4477 - sparse_categorical_crossentropy: 0.6934 - val_accuracy: 0.5159 - val_loss: 1.2921 - val_sparse_categorical_crossentropy: 0.6931
Epoch 3/100
.55/55 - 2s - 36ms/step - accuracy: 0.5010 - loss: 1.1802 - sparse_categorical_crossentropy: 0.6936 - val_accuracy: 0.5176 - val_loss: 1.0762 - val_sparse_categorical_crossentropy: 0.6928
Epoch 4/100
.55/55 - 2s - 35ms/step - accuracy: 0.4990 - loss: 1.0029 - sparse_categorical_crossentropy: 0.6930 - val_accuracy: 0.5176 - val_loss: 0.9359 - val_sparse_categorical_crossentropy: 0.6929
Epoch 5/100
.55/55 - 2s - 

KeyboardInterrupt: 

In [18]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

In [33]:
def test_deepnn_for_window(window_size=249):
  all_series = []
  all_labels = []
  loader = TimeSeriesDataLoader(window_size=window_size)
  for i in range (0,10000,1):
    series_data,y_value,boundary = loader.get_series_by_id(i,True)
    series_data_clean = series_data["value"]
    series_data_clean = series_data_clean[boundary-10:] ## trimmt so, dass der Anfang 10 Timesteps vor dem boundary point ist
    all_series.append(series_data_clean)
    all_labels.append(y_value)
  data_x = np.array(all_series)  # Shape: (num_ids, 498)
  labels = np.array(all_labels)   # Shape: (num_ids,)

  print(f"Data shape: {data_x.shape}")
  print(f"Labels shape: {labels.shape}")

  true_indices = np.where(labels == True)[0]
  false_indices = np.where(labels == False)[0]

  print(f"True samples: {len(true_indices)}")   # ~3333
  print(f"False samples: {len(false_indices)}") # ~6666

  from sklearn.utils import resample

  false_balanced = resample(false_indices,
                          replace=False,
                          n_samples=len(true_indices),
                          random_state=42)

  # Combine balanced indices
  balanced_indices = np.concatenate([true_indices, false_balanced])
  np.random.shuffle(balanced_indices)

  # Create balanced dataset by indexing your existing arrays
  data_x_balanced = data_x[balanced_indices]
  labels_balanced = labels[balanced_indices].astype(int)  # Convert True/False to 1/0

  print(f"Balanced data shape: {data_x_balanced.shape}")  # Should be (~6666, 498)
  print(f"Balanced labels shape: {labels_balanced.shape}")
  print(f"Balanced class distribution: {np.bincount(labels_balanced)}")

  # Now use data_x_balanced and labels_balanced for your model
  data_x = data_x_balanced
  labels = labels_balanced

  # Method 1: Per-sample min-max normalization (RECOMMENDED)
  # This preserves the shape of each time series while normalizing to [0,1]
  def normalize_per_sample(data):
      normalized = np.zeros_like(data)
      for i in range(data.shape[0]):
          sample = data[i]
          sample_min = sample.min()
          sample_max = sample.max()
          if sample_max > sample_min:  # Avoid division by zero
              normalized[i] = (sample - sample_min) / (sample_max - sample_min)
          else:
              normalized[i] = np.zeros_like(sample)  # Flat signal = all zeros
      return normalized

  # Apply the better normalization
  data_x_norm = normalize_per_sample(data_x_balanced)

  # Check if this fixes the variation issue
  print("After per-sample normalization:")
  print(f"First sample std: {data_x_norm[0].std():.4f}")
  print(f"Sample means: {data_x_norm.mean(axis=1)[:5]}")
  print(f"Sample stds: {data_x_norm.std(axis=1)[:5]}")
  print(f"Overall range: [{data_x_norm.min():.4f}, {data_x_norm.max():.4f}]")

  # Split and train
  x_train, x_test, y_train, y_test = train_test_split(
      data_x_norm, labels_balanced.reshape(-1, 1), train_size=0.8, random_state=42
  )

  x_train = Transform2D2TR(x_train, rescale=False, times=3)
  x_test = Transform2D2TR(x_test, rescale=False, times=3)

"""
  learning_rate = 5e-5
  epochs = 300
  batch_size = 64
  dropout_rate = 0.3
  n_filter = 16
  n = x_train.shape[-1]
  num_tran = x_train.shape[1]
  kernel_size = (num_tran // 2, 10)
  num_classes = 2
  logdir = Path("tensorboard_logs", "Trial")
  # %%
  num_resblock = 3
  model_name = "Toshko"
  print(model_name)
  # build the model
  m = np.array([50, 40, 30, 20, 10])
  model = general_deep_nn(
      n,
      num_tran,
      kernel_size,
      n_filter,
      dropout_rate,
      num_classes,
      num_resblock,
      m,
      5,
      model_name=model_name,
  )
  model.summary()

  size_histories = {}
  epochdots = tfdoc_model.EpochDots()
  size_histories[model_name] = compile_and_fit(
      model,
      x_train,
      y_train,
      batch_size,
      learning_rate,
      model_name,
      logdir,
      epochdots,
      validation_split=0.25,
      max_epochs=epochs,
  )
  plotter = tfdoc_plot.HistoryPlotter(metric="accuracy", smoothing_std=10)
  plt.figure(figsize=(10, 8))
  plotter.plot(size_histories)
  acc_name = model_name + "+acc.png"
  acc_path = Path(logdir, model_name, acc_name)
  plt.savefig(acc_path)


  model_path = Path(logdir, model_name, "model")
 # model.save(model_path)

  #  Confusion Matrix
  model_pred = model.evaluate(x_test, y_test, verbose=2)
  y_prob = np.max(model.predict(x_test), axis=1)
  y_pred = np.argmax(model.predict(x_test), axis=1)
  confusion_mtx = tf.math.confusion_matrix(y_test, y_pred)
  plt.figure(figsize=(10, 8))
  label_vec = np.arange(num_classes)
  sns.heatmap(
      confusion_mtx,
      cmap="YlGnBu",
      xticklabels=label_vec,
      yticklabels=label_vec,
      annot=True,
      fmt="g",
  )
  plt.xlabel("Prediction")
  plt.ylabel("Label")

  # Calculate ROC AUC
  roc_auc = roc_auc_score(y_test, y_pred)
  print(f"ROC AUC Score: {roc_auc:.4f}")
  """

'\n  learning_rate = 5e-5\n  epochs = 300\n  batch_size = 64\n  dropout_rate = 0.3\n  n_filter = 16\n  n = x_train.shape[-1]\n  num_tran = x_train.shape[1]\n  kernel_size = (num_tran // 2, 10)\n  num_classes = 2\n  logdir = Path("tensorboard_logs", "Trial")\n  # %%\n  num_resblock = 3\n  model_name = "Toshko"\n  print(model_name)\n  # build the model\n  m = np.array([50, 40, 30, 20, 10])\n  model = general_deep_nn(\n      n,\n      num_tran,\n      kernel_size,\n      n_filter,\n      dropout_rate,\n      num_classes,\n      num_resblock,\n      m,\n      5,\n      model_name=model_name,\n  )\n  model.summary()\n\n  size_histories = {}\n  epochdots = tfdoc_model.EpochDots()\n  size_histories[model_name] = compile_and_fit(\n      model,\n      x_train,\n      y_train,\n      batch_size,\n      learning_rate,\n      model_name,\n      logdir,\n      epochdots,\n      validation_split=0.25,\n      max_epochs=epochs,\n  )\n  plotter = tfdoc_plot.HistoryPlotter(metric="accuracy", smoothing_

In [34]:
for window_size in [10,20,30,50,100]:
  test_deepnn_for_window(window_size)

Loading X data from data/X_train.parquet...
Loading y data from data/y_train.parquet...
Loaded 10001 time series


Exception ignored in: <function _xla_gc_callback at 0x7e26683763e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


Data shape: (10000, 19)
Labels shape: (10000,)
True samples: 2908
False samples: 7092
Balanced data shape: (5816, 19)
Balanced labels shape: (5816,)
Balanced class distribution: [2908 2908]
After per-sample normalization:
First sample std: 0.2671
Sample means: [0.49843171 0.49099493 0.64992319 0.28860509 0.51416152]
Sample stds: [0.2670732  0.27057148 0.21044334 0.19730819 0.25259211]
Overall range: [0.0000, 1.0000]
Loading X data from data/X_train.parquet...
Loading y data from data/y_train.parquet...
Loaded 10001 time series
Data shape: (10000, 29)
Labels shape: (10000,)
True samples: 2908
False samples: 7092
Balanced data shape: (5816, 29)
Balanced labels shape: (5816,)
Balanced class distribution: [2908 2908]
After per-sample normalization:
First sample std: 0.2567
Sample means: [0.46123719 0.40825061 0.53270136 0.59324478 0.5670513 ]
Sample stds: [0.25674367 0.30650708 0.24781146 0.2337478  0.22835682]
Overall range: [0.0000, 1.0000]
Loading X data from data/X_train.parquet...
Loa

KeyboardInterrupt: 

In [35]:
import numpy as np
import tensorflow as tf
from keras import layers, models, losses, metrics
from sklearn.utils import shuffle
import warnings
class FocusedChangePointDetector:
  """
  A changepoint detector that focuses on determining if a specific point
  in a time series is a changepoint or not.
  """
  def __init__(self, window_size=50, n_transformations=4):
      """
      Parameters
      ----------
      window_size : int
          Size of the window around the target point (total window = 2 * window_size)
      n_transformations : int
          Number of transformations to apply to the time series
      """
      self.window_size = window_size
      self.n_transformations = n_transformations
      self.model = None

  def create_centered_windows(self, time_series, target_indices, window_size=None):
      """
      Create windows centered around specific target points.

      Parameters
      ----------
      time_series : np.ndarray
          Shape (n,) or (batch_size, n) - the time series data
      target_indices : np.ndarray
          Indices of points to check for changepoints
      window_size : int
          Half-window size (full window will be 2 * window_size)

      Returns
      -------
      np.ndarray
          Windows centered around target points
      """
      if window_size is None:
          window_size = self.window_size

      if len(time_series.shape) == 1:
          time_series = time_series.reshape(1, -1)

      batch_size = time_series.shape[0]
      n = time_series.shape[1]

      # Ensure indices are arrays
      if np.isscalar(target_indices):
          target_indices = np.array([target_indices])

      windows = []
      valid_indices = []

      for b in range(batch_size):
          for idx in target_indices:
              # Check if we can create a full window
              if idx - window_size >= 0 and idx + window_size <= n:
                  window = time_series[b, idx - window_size : idx + window_size]
                  windows.append(window)
                  valid_indices.append(idx)

      return np.array(windows), np.array(valid_indices)

  def transform_windows(self, windows, rescale=True):
      """
      Apply transformations to windows (similar to Transform2D but adapted).

      Parameters
      ----------
      windows : np.ndarray
          Shape (n_windows, window_length)
      rescale : bool
          Whether to rescale the transformations

      Returns
      -------
      np.ndarray
          Shape (n_windows, n_transformations, window_length)
      """
      n_windows, length = windows.shape
      windows[windows == 0.0] = 1e-8

      # Original
      original = windows.copy()

      # Squared
      squared = np.square(windows)

      # Log squared
      log_squared = np.log(np.abs(squared) + 1e-8)

      # Tanh (or cumsum for change detection)
      # Using difference to highlight changes
      diff = np.diff(windows, axis=1, prepend=windows[:, 0:1])
      tanh_transform = np.tanh(diff)

      if rescale:
          # Rescale each transformation
          original = self._rescale_array(original)
          squared = self._rescale_array(squared)
          log_squared = self._rescale_array(log_squared)
          tanh_transform = self._rescale_array(tanh_transform)

      # Stack transformations
      result = np.stack([original, squared, log_squared, tanh_transform], axis=1)
      return result

  def _rescale_array(self, arr):
      """Rescale array to [0, 1] range."""
      arr_min = np.min(arr, axis=1, keepdims=True)
      arr_max = np.max(arr, axis=1, keepdims=True)
      arr_range = arr_max - arr_min
      arr_range[arr_range == 0] = 1.0
      return (arr - arr_min) / arr_range

  def build_focused_model(self, input_length=None):
      """
      Build a CNN model focused on the center point of the window.

      Parameters
      ----------
      input_length : int
          Length of the input window (default: 2 * window_size)

      Returns
      -------
      keras.Model
          The compiled model
      """
      if input_length is None:
          input_length = 2 * self.window_size

      input_layer = layers.Input(shape=(self.n_transformations, input_length))

      # Reshape for Conv2D
      x = layers.Reshape((self.n_transformations, input_length, 1))(input_layer)

      # First conv block with attention on center
      x = layers.Conv2D(32, (2, 5), padding='same', activation='relu')(x)
      x = layers.BatchNormalization()(x)

      # Add positional weighting to emphasize center
      position_weights = self._create_position_weights(input_length)
      x = layers.Lambda(lambda z: z * position_weights)(x)

      x = layers.Conv2D(64, (2, 5), padding='same', activation='relu')(x)
      x = layers.BatchNormalization()(x)
      x = layers.MaxPooling2D((1, 2))(x)

      # Second conv block
      x = layers.Conv2D(128, (2, 3), padding='same', activation='relu')(x)
      x = layers.BatchNormalization()(x)
      x = layers.MaxPooling2D((1, 2))(x)

      # Third conv block focusing on center region
      x = layers.Conv2D(256, (2, 3), padding='same', activation='relu')(x)
      x = layers.BatchNormalization()(x)

      # Global pooling and dense layers
      x = layers.GlobalAveragePooling2D()(x)
      x = layers.Dense(128, activation='relu')(x)
      x = layers.Dropout(0.5)(x)
      x = layers.Dense(64, activation='relu')(x)
      x = layers.Dropout(0.3)(x)

      # Output layer (binary classification)
      output_layer = layers.Dense(1, activation='sigmoid')(x)

      model = models.Model(input_layer, output_layer, name='focused_cpd')

      model.compile(
          optimizer='adam',
          loss='binary_crossentropy',
          metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
      )

      self.model = model
      return model

  def _create_position_weights(self, length):
      """
      Create position weights that emphasize the center of the window.
      """
      # Gaussian-like weights centered at the middle
      center = length / 2
      positions = np.arange(length)
      weights = np.exp(-0.5 * ((positions - center) / (length / 6)) ** 2)
      weights = weights / np.max(weights)
      return tf.constant(weights.reshape(1, 1, length, 1), dtype=tf.float32)

  def generate_training_data(self, n_samples=1000, series_length=200,
                          changepoint_prob=0.5, noise_level=0.1):
      """
      Generate synthetic training data with known changepoints at specific positions.

      Parameters
      ----------
      n_samples : int
          Number of training samples
      series_length : int
          Length of each time series
      changepoint_prob : float
          Probability of having a changepoint
      noise_level : float
          Noise level in the data

      Returns
      -------
      tuple
          (X_train, y_train, changepoint_positions)
      """
      X_data = []
      y_labels = []
      cp_positions = []

      for _ in range(n_samples):
          # Generate base time series
          ts = np.zeros(series_length)

          # Randomly decide if there's a changepoint
          has_cp = np.random.rand() < changepoint_prob

          # Choose a target position (center region to ensure full window)
          target_pos = np.random.randint(self.window_size,
                                        series_length - self.window_size)

          if has_cp:
              # Create a changepoint at the target position
              mean_before = np.random.randn()
              mean_after = mean_before + np.random.uniform(0.5, 2.0) * np.random.choice([-1, 1])

              ts[:target_pos] = mean_before + np.random.randn(target_pos) * noise_level
              ts[target_pos:] = mean_after + np.random.randn(series_length - target_pos) * noise_level

              label = 1
          else:
              # No changepoint - smooth series
              ts = np.random.randn() + np.random.randn(series_length) * noise_level
              # Add some smooth variation
              ts = ts + np.sin(np.linspace(0, 4*np.pi, series_length)) * 0.5
              label = 0

          # Extract window around target position
          window = ts[target_pos - self.window_size : target_pos + self.window_size]

          X_data.append(window)
          y_labels.append(label)
          cp_positions.append(target_pos)

      X_data = np.array(X_data)
      y_labels = np.array(y_labels)

      # Apply transformations
      X_transformed = self.transform_windows(X_data)

      return X_transformed, y_labels, np.array(cp_positions)

  def train(self, X_train, y_train, validation_split=0.2,
            epochs=50, batch_size=32, verbose=1):
      """
      Train the model.

      Parameters
      ----------
      X_train : np.ndarray
          Training data (already transformed)
      y_train : np.ndarray
          Training labels
      validation_split : float
          Validation split ratio
      epochs : int
          Number of epochs
      batch_size : int
          Batch size
      verbose : int
          Verbosity level

      Returns
      -------
      History
          Training history
      """
      if self.model is None:
          self.build_focused_model()

      # Shuffle data
      X_train, y_train = shuffle(X_train, y_train, random_state=42)

      # Add callbacks
      callbacks = [
          tf.keras.callbacks.EarlyStopping(
              monitor='val_loss',
              patience=10,
              restore_best_weights=True
          ),
          tf.keras.callbacks.ReduceLROnPlateau(
              monitor='val_loss',
              factor=0.5,
              patience=5,
              min_lr=1e-6
          )
      ]

      history = self.model.fit(
          X_train, y_train,
          validation_split=validation_split,
          epochs=epochs,
          batch_size=batch_size,
          callbacks=callbacks,
          verbose=verbose
      )

      return history

  def predict_point(self, time_series, target_index):
      """
      Predict if a specific point is a changepoint.

      Parameters
      ----------
      time_series : np.ndarray
          The time series (1D array)
      target_index : int
          The index to check for changepoint

      Returns
      -------
      dict
          Dictionary with prediction, probability, and validity
      """
      # Check if we can create a valid window
      n = len(time_series)
      if target_index - self.window_size < 0 or target_index + self.window_size > n:
          return {
              'valid': False,
              'message': f'Cannot create window around index {target_index}',
              'prediction': None,
              'probability': None
          }

      # Extract window
      window = time_series[target_index - self.window_size :
                          target_index + self.window_size]

      # Transform window
      window_transformed = self.transform_windows(window.reshape(1, -1))

      # Predict
      prob = self.model.predict(window_transformed, verbose=0)[0, 0]
      prediction = int(prob > 0.5)

      return {
          'valid': True,
          'prediction': prediction,
          'probability': float(prob),
          'is_changepoint': bool(prediction)
      }

  def evaluate_multiple_points(self, time_series, indices):
      """
      Evaluate multiple points in a time series.

      Parameters
      ----------
      time_series : np.ndarray
          The time series
      indices : list or np.ndarray
          Indices to evaluate

      Returns
      -------
      list
          List of prediction dictionaries
      """
      results = []
      for idx in indices:
          result = self.predict_point(time_series, idx)
          result['index'] = idx
          results.append(result)
      return results

In [36]:
detector = FocusedChangePointDetector(window_size=50)

In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
class DataPreparationPipeline:
    """
    Prepare data from your loader for the FocusedChangePointDetector.
    """
    def __init__(self, loader, window_size=50, n_transformations=4):
        """
        Parameters
        ----------
        loader : object
            Your data loader object with get_series_by_id method
        window_size : int
            Half-window size for the model (full window = 2 * window_size)
        n_transformations : int
            Number of transformations to apply
        """
        self.loader = loader
        self.window_size = window_size
        self.n_transformations = n_transformations
        self.detector = None  # Will hold the FocusedChangePointDetector

    def prepare_single_series(self, series_id, include_negatives=True, negative_ratio=1.0):
        """
        Prepare data from a single series.

        Parameters
        ----------
        series_id : int
            ID of the series to load
        include_negatives : bool
            Whether to include non-changepoint samples
        negative_ratio : float
            Ratio of negative samples to include (relative to 1 positive sample)

        Returns
        -------
        tuple
            (windows, labels, metadata)
        """
        # Load data using your loader
        series_data, y_value, boundary = self.loader.get_series_by_id(series_id, True)

        # Remove the 'period' column if it exists and get just the values
        if isinstance(series_data, pd.DataFrame):
            if 'period' in series_data.columns:
                series_data = series_data.drop('period', axis=1)
            # Assuming the main value column is the first one after dropping 'period'
            # Adjust this based on your actual column names
            values = series_data.iloc[:, 0].values if series_data.shape[1] == 1 else series_data.values.flatten()
        else:
            values = np.array(series_data).flatten()

        windows = []
        labels = []
        indices = []

        # The boundary point is a changepoint (positive sample)
        if self._can_create_window(boundary, len(values)):
            window = self._extract_window(values, boundary)
            windows.append(window)
            labels.append(y_value)  # Using y_value from your loader
            indices.append(boundary)

        # Generate negative samples (non-changepoint windows)
        if include_negatives and negative_ratio > 0:
            negative_windows, negative_indices = self._generate_negative_samples(
                values, boundary, int(negative_ratio)
            )
            windows.extend(negative_windows)
            labels.extend([0] * len(negative_windows))  # 0 for non-changepoints
            indices.extend(negative_indices)

        if len(windows) == 0:
            return None, None, None

        # Convert to numpy arrays
        windows = np.array(windows)
        labels = np.array(labels)

        # Apply transformations
        windows_transformed = self._transform_windows(windows)

        metadata = {
            'series_id': series_id,
            'boundary': boundary,
            'indices': indices,
            'original_length': len(values)
        }

        return windows_transformed, labels, metadata

    def prepare_multiple_series(self, series_ids=None, n_series=None,
                              include_negatives=True, negative_ratio=1.0):
        """
        Prepare data from multiple series.

        Parameters
        ----------
        series_ids : list
            List of series IDs to load. If None, will use range(n_series)
        n_series : int
            Number of series to load if series_ids is None
        include_negatives : bool
            Whether to include non-changepoint samples
        negative_ratio : float
            Ratio of negative samples per positive sample

        Returns
        -------
        tuple
            (X_all, y_all, metadata_all)
        """
        if series_ids is None:
            if n_series is None:
                raise ValueError("Either series_ids or n_series must be provided")
            series_ids = range(n_series)

        all_windows = []
        all_labels = []
        all_metadata = []

        for i, sid in enumerate(series_ids):
            try:
                windows, labels, metadata = self.prepare_single_series(
                    sid, include_negatives, negative_ratio
                )

                if windows is not None:
                    all_windows.append(windows)
                    all_labels.append(labels)
                    all_metadata.append(metadata)

                if (i + 1) % 100 == 0:
                    print(f"Processed {i + 1}/{len(series_ids)} series")

            except Exception as e:
                print(f"Error processing series {sid}: {e}")
                continue

        if len(all_windows) == 0:
            raise ValueError("No valid windows could be created from the data")

        # Concatenate all data
        X_all = np.concatenate(all_windows, axis=0)
        y_all = np.concatenate(all_labels, axis=0)

        print(f"\nData preparation complete:")
        print(f"Total samples: {len(X_all)}")
        print(f"Positive samples (changepoints): {np.sum(y_all == 1)}")
        print(f"Negative samples (non-changepoints): {np.sum(y_all == 0)}")
        print(f"Data shape: {X_all.shape}")

        return X_all, y_all, all_metadata

    def prepare_train_test_split(self, series_ids=None, n_series=None,
                                test_size=0.2, include_negatives=True,
                                negative_ratio=1.0, random_state=42):
        """
        Prepare data and split into train/test sets.

        Parameters
        ----------
        series_ids : list
            List of series IDs to load
        n_series : int
            Number of series to load if series_ids is None
        test_size : float
            Proportion of data for testing
        include_negatives : bool
            Whether to include non-changepoint samples
        negative_ratio : float
            Ratio of negative samples per positive sample
        random_state : int
            Random seed for reproducibility

        Returns
        -------
        tuple
            (X_train, X_test, y_train, y_test)
        """
        # Prepare all data
        X_all, y_all, metadata = self.prepare_multiple_series(
            series_ids, n_series, include_negatives, negative_ratio
        )

        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(
            X_all, y_all, test_size=test_size, random_state=random_state,
            stratify=y_all  # Maintain class balance in splits
        )

        print(f"\nTrain/Test split:")
        print(f"Training samples: {len(X_train)} (Positive: {np.sum(y_train == 1)})")
        print(f"Testing samples: {len(X_test)} (Positive: {np.sum(y_test == 1)})")

        return X_train, X_test, y_train, y_test

    def prepare_for_prediction(self, series_id, target_index=None):
        """
        Prepare a single series for prediction at a specific point.

        Parameters
        ----------
        series_id : int
            ID of the series to load
        target_index : int
            Specific index to check. If None, uses the boundary point

        Returns
        -------
        tuple
            (window_transformed, target_index, original_values)
        """
        # Load data
        series_data, y_value, boundary = self.loader.get_series_by_id(series_id, True)

        # Process values
        if isinstance(series_data, pd.DataFrame):
            if 'period' in series_data.columns:
                series_data = series_data.drop('period', axis=1)
            values = series_data.iloc[:, 0].values if series_data.shape[1] == 1 else series_data.values.flatten()
        else:
            values = np.array(series_data).flatten()

        # Use boundary as target if not specified
        if target_index is None:
            target_index = boundary

        # Check if we can create a window
        if not self._can_create_window(target_index, len(values)):
            raise ValueError(f"Cannot create window around index {target_index}")

        # Extract and transform window
        window = self._extract_window(values, target_index)
        window_transformed = self._transform_windows(np.array([window]))

        return window_transformed, target_index, values

    def _can_create_window(self, index, series_length):
        """Check if a valid window can be created around the index."""
        return index >= self.window_size and index + self.window_size <= series_length

    def _extract_window(self, values, center_index):
        """Extract a window centered at the given index."""
        start = center_index - self.window_size
        end = center_index + self.window_size
        return values[start:end]

    def _generate_negative_samples(self, values, boundary, n_negatives):
        """
        Generate negative samples (non-changepoint windows) from the series.

        Parameters
        ----------
        values : np.ndarray
            The time series values
        boundary : int
            The actual changepoint location to avoid
        n_negatives : int
            Number of negative samples to generate

        Returns
        -------
        tuple
            (windows, indices)
        """
        series_length = len(values)
        windows = []
        indices = []

        # Define safe zones (away from the actual boundary)
        safe_distance = self.window_size // 2  # Minimum distance from boundary

        # Potential negative sample positions
        potential_indices = []
        for idx in range(self.window_size, series_length - self.window_size):
            if abs(idx - boundary) > safe_distance:
                potential_indices.append(idx)

        if len(potential_indices) == 0:
            return windows, indices

        # Sample negative indices
        n_to_sample = min(n_negatives, len(potential_indices))
        selected_indices = np.random.choice(potential_indices, n_to_sample, replace=False)

        for idx in selected_indices:
            window = self._extract_window(values, idx)
            windows.append(window)
            indices.append(idx)

        return windows, indices

    def _transform_windows(self, windows):
        """Apply transformations to windows."""
        if len(windows.shape) == 1:
            windows = windows.reshape(1, -1)

        n_windows, length = windows.shape
        windows[windows == 0.0] = 1e-8

        # Original
        original = windows.copy()

        # Squared
        squared = np.square(windows)

        # Log squared
        log_squared = np.log(np.abs(squared) + 1e-8)

        # Difference (to highlight changes)
        diff = np.diff(windows, axis=1, prepend=windows[:, 0:1])
        tanh_transform = np.tanh(diff)

        # Rescale each transformation
        original = self._rescale_array(original)
        squared = self._rescale_array(squared)
        log_squared = self._rescale_array(log_squared)
        tanh_transform = self._rescale_array(tanh_transform)

        # Stack transformations
        result = np.stack([original, squared, log_squared, tanh_transform], axis=1)
        return result

    def _rescale_array(self, arr):
        """Rescale array to [0, 1] range."""
        arr = arr.copy()
        arr_min = np.min(arr, axis=1, keepdims=True)
        arr_max = np.max(arr, axis=1, keepdims=True)
        arr_range = arr_max - arr_min
        arr_range[arr_range == 0] = 1.0
        return (arr - arr_min) / arr_range

In [41]:
prep = DataPreparationPipeline(loader, window_size=50)

# Prepare data from multiple series
X_train, X_test, y_train, y_test = prep.prepare_train_test_split(
    n_series=1000,  # Use 1000 series for training
    test_size=0.2,
    include_negatives=True,  # Include non-changepoint samples
    negative_ratio=2.0  # 2 negative samples per positive
)

# Train the model (using the FocusedChangePointDetector from before)
detector = FocusedChangePointDetector(window_size=50)
detector.train(X_train, y_train)


Processed 100/1000 series
Processed 200/1000 series
Processed 300/1000 series
Processed 400/1000 series
Processed 500/1000 series
Processed 600/1000 series
Processed 700/1000 series
Processed 800/1000 series
Processed 900/1000 series
Processed 1000/1000 series

Data preparation complete:
Total samples: 3000
Positive samples (changepoints): 283
Negative samples (non-changepoints): 2717
Data shape: (3000, 4, 100)

Train/Test split:
Training samples: 2400 (Positive: 226)
Testing samples: 600 (Positive: 57)
Epoch 1/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 12s 32ms/step - accuracy: 0.8611 - auc: 0.5764 - loss: 0.4285 - val_accuracy: 0.9271 - val_auc: 0.4902 - val_loss: 0.2807 - learning_rate: 0.0010
Epoch 2/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9125 - auc: 0.5594 - loss: 0.3206 - val_accuracy: 0.9271 - val_auc: 0.5132 - val_loss: 0.2738 - learning_rate: 0.0010
Epoch 3/50
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8982 - auc: 0.6295 - loss: 0.3271 - val_accuracy: 0.9271 - va

In [42]:
# Predict on a new series
window, target_idx, values = prep.prepare_for_prediction(series_id=1001)
prob = detector.model.predict(window)[0, 0]
print(f"Changepoint probability: {prob:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Changepoint probability: 0.0585


## Understanding the Data

The dataset consists of univariate time series, each containing ~2,000-5,000 values with a designated boundary point. For each time series, you need to determine whether a structural break occurred at this boundary point.

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

In [8]:
# Load the data simply
X_train, y_train, X_test = crunch.load_data()

data/X_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_train.parquet (204327238 bytes)
data/X_train.parquet: already exists, file length match
data/X_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/X_test.reduced.parquet (2380918 bytes)
data/X_test.reduced.parquet: already exists, file length match
data/y_train.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_train.parquet (61003 bytes)
data/y_train.parquet: already exists, file length match
data/y_test.reduced.parquet: download from https:crunchdao--competition--production.s3-accelerate.amazonaws.com/data-releases/146/y_test.reduced.parquet (2655 bytes)
data/y_test.reduced.parquet: already exists, file length match


### Understanding `X_train`

The training data is structured as a pandas DataFrame with a MultiIndex:

**Index Levels:**
- `id`: Identifies the unique time series
- `time`: The timestep within each time series

**Columns:**
- `value`: The actual time series value at each timestep
- `period`: A binary indicator where `0` represents the **period before** the boundary point, and `1` represents the **period after** the boundary point

In [ ]:
X_train

value  period
id    time                  
0     0     0.001858       0
      1    -0.001664       0
      2    -0.004386       0
      3     0.000699       0
      4    -0.002433       0
...              ...     ...
10000 1890 -0.005903       1
      1891  0.007295       1
      1892  0.003527       1
      1893  0.007218       1
      1894  0.000034       1

[23802099 rows x 2 columns]

### Understanding `y_train`

This is a simple `pandas.Series` that tells if a dataset id has a structural breakpoint or not.

**Index:**
- `id`: the ID of the dataset

**Value:**
- `structural_breakpoint`: Boolean indicating whether a structural break occurred (`True`) or not (`False`)

In [ ]:
y_train

id
0         True
1         True
2        False
3         True
4        False
         ...  
9996     False
9997      True
9998     False
9999     False
10000     True
Name: structural_breakpoint, Length: 10001, dtype: bool

### Understanding `X_test`

The test data is provided as a **`list` of `pandas.DataFrame`s** with the same format as [`X_train`](#understanding-X_test).

It is structured as a list to encourage processing records one by one, which will be mandatory in the `infer()` function.

In [ ]:
print("Number of datasets:", len(X_test))

Number of datasets: 101


In [ ]:
X_test[0]

value  period
id    time                  
10001 0    -0.020657       0
      1    -0.005894       0
      2    -0.003052       0
      3    -0.000590       0
      4     0.009887       0
...              ...     ...
      2517  0.005084       1
      2518 -0.024414       1
      2519 -0.014986       1
      2520  0.012999       1
      2521 -0.022138       1

[2522 rows x 2 columns]

## Strategy Implementation

There are multiple approaches you can take to detect structural breaks:

1. **Statistical Tests**: Compare distributions before and after the boundary point;
2. **Feature Engineering**: Extract features from both segments for comparison;
3. **Time Series Modeling**: Detect deviations from expected patterns;
4. **Machine Learning**: Train models to recognize break patterns from labeled examples.

The baseline implementation below uses a simple statistical approach: a t-test to compare the distributions before and after the boundary point.

### The `train()` Function

In this function, you build and train your model for making inferences on the test data. Your model must be stored in the `model_directory_path`.

The baseline implementation below doesn't require a pre-trained model, as it uses a statistical test that will be computed at inference time.

In [ ]:
def train(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    model_directory_path: str,
):
    # For our baseline t-test approach, we don't need to train a model
    # This is essentially an unsupervised approach calculated at inference time
    model = None

    # You could enhance this by training an actual model, for example:
    # 1. Extract features from before/after segments of each time series
    # 2. Train a classifier using these features and y_train labels
    # 3. Save the trained model

    joblib.dump(model, os.path.join(model_directory_path, 'model.joblib'))

### The `infer()` Function

In the inference function, your trained model (if any) is loaded and used to make predictions on test data.

**Important workflow:**
1. Load your model;
2. Use the `yield` statement to signal readiness to the runner;
3. Process each dataset one by one within the for loop;
4. For each dataset, use `yield prediction` to return your prediction.

**Note:** The datasets can only be iterated once!

In [ ]:
def infer(
    X_test: typing.Iterable[pd.DataFrame],
    model_directory_path: str,
):
    model = joblib.load(os.path.join(model_directory_path, 'model.joblib'))

    yield  # Mark as ready

    # X_test can only be iterated once.
    # Before getting the next dataset, you must predict the current one.
    for dataset in X_test:
        # Baseline approach: Compute t-test between values before and after boundary point
        # The negative p-value is used as our score - smaller p-values (larger negative numbers)
        # indicate more evidence against the null hypothesis that distributions are the same,
        # suggesting a structural break
        def t_test(u: pd.DataFrame):
            return -scipy.stats.ttest_ind(
                u["value"][u["period"] == 0],  # Values before boundary point
                u["value"][u["period"] == 1],  # Values after boundary point
            ).pvalue

        prediction = t_test(dataset)
        yield prediction  # Send the prediction for the current dataset

        # Note: This baseline approach uses a t-test to compare the distributions
        # before and after the boundary point. A smaller p-value (larger negative number)
        # suggests stronger evidence that the distributions are different,
        # indicating a potential structural break.

## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [ ]:
crunch.test(
    # Uncomment to disable the train
    # force_first_train=False,

    # Uncomment to disable the determinism check
    # no_determinism_check=True,
)

## Results

Once the local tester is done, you can preview the result stored in `data/prediction.parquet`.

In [ ]:
prediction = pd.read_parquet("data/prediction.parquet")
prediction

### Local scoring

You can call the function that the system uses to estimate your score locally.

In [ ]:
# Load the targets
target = pd.read_parquet("data/y_test.reduced.parquet")["structural_breakpoint"]

# Call the scoring function
sklearn.metrics.roc_auc_score(
    target,
    prediction,
)

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/structural-break/submit/notebook

![Download and Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/download-and-submit-notebook.gif)